In [17]:
import pandas as pd
from bs4 import BeautifulSoup

In [18]:
df_original = pd.read_csv("hackernews.csv", parse_dates=["timestamp"])
df_original

,title,url,text,dead,by,score,time,timestamp,type,id,parent,descendants,ranking,deleted
0,NaN,NaN,The worst case for kickstarter is that they fa...,NaN,AJ007,NaN,1347981310,2012-09-18 15:15:10+00:00,comment,4538506,4537746.0,NaN,NaN,NaN
1,NaN,NaN,I&#x27;d have to say the same for Facebook. Mo...,NaN,jorgecurio,NaN,1456001084,2016-02-20 20:44:44+00:00,comment,11141867,11141158.0,NaN,NaN,NaN
2,NaN,NaN,&#62; You can't get San Francisco running effi...,NaN,Cushman,NaN,1359100263,2013-01-25 07:51:03+00:00,comment,5114503,5114256.0,NaN,NaN,NaN
3,NaN,NaN,&gt;illegal to transmit encrypted data over th...,NaN,aoeuasdf,NaN,1633726031,2021-10-08 20:47:11+00:00,comment,28803994,28803811.0,NaN,NaN,NaN
4,NaN,NaN,I actually think ARkit will be big precisely b...,NaN,erikpukinskis,NaN,1501892529,2017-08-05 00:22:09+00:00,comment,14933475,14932842.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4802,NaN,NaN,Or split into steam-the-platform-the-company a...,NaN,jrkatz,NaN,1564069670,2019-07-25 15:47:50+00:00,comment,20526386,20525938.0,NaN,NaN,NaN
4803,NaN,NaN,This isn&#x27;t as serious of a proposal as so...,NaN,yajoe,NaN,1387069124,2013-12-15 00:58:44+00:00,comment,6907828,6907262.0,NaN,NaN,NaN
4804,NaN,NaN,Ex coworkers. Last time I got a job tradition...,NaN,VLM,NaN,1378900686,2013-09-11 11:58:06+00:00,comment,6366593,6365783.0,NaN,NaN,NaN
4805,NaN,NaN,I predict that 0.10.x will be with us for a lo...,NaN,erichocean,NaN,1389925140,2014-01-17 02:19:00+00:00,comment,7073895,7073044.0,NaN,NaN,NaN


In [19]:
def extract_prediction(text):
    soup = BeautifulSoup(text)
    text = soup.text
    text = " ".join(text.split())
    pred_idx = text.find("I predict")
    dot_idx = text[pred_idx:].find(".")

    return text[pred_idx : pred_idx + dot_idx + 1]

In [20]:
df = df_original.copy()
df["prediction"] = df["text"].apply(extract_prediction)
df["year"] = df["timestamp"].dt.year
df = df[df["prediction"] != ""]

In [21]:
# Original extractor was too simple
# Patching dataframe using a new extractor, but without changing the order of the predictions to keep the JSON IDs
from collections import Counter
import re

def extract_prediction_regex(text):
    text = " ".join(text.split())
    text = text.replace("<p>", ". ").replace("i.e.", "ie").replace("e.g.", "eg")
    soup = BeautifulSoup(text)
    text = soup.text
    regex = re.compile(
        r"I predict .*?(?:[.!?](?=\s|\.|.\s)|$)"
    )  # https://stackoverflow.com/questions/73355650/how-to-extract-a-sentence-with-start-marker/73355899#73355899
    matches = regex.findall(text)

    # Edge cases
    if matches:
        prediction = matches[0]
        if prediction.endswith("?") or prediction.endswith(":.") or Counter(prediction)["\""] % 2 != 0:
            return ""
        if (prediction.endswith(").") or prediction.endswith(".)")) and "(" not in prediction:
            return prediction[:-2] + "."
        
        return prediction
    else:
        return ""

df2 = df_original.copy()
df2["prediction"] = df2["text"].apply(extract_prediction_regex)
df2["year"] = df2["timestamp"].dt.year
df2 = df2[df2["prediction"] != ""]
df = df.merge(df2, how="left", on="id", suffixes=(None, "_new"))
df.loc[~df["prediction_new"].isna(), "prediction"] = df.loc[~df["prediction_new"].isna()]["prediction_new"] 
df = pd.concat([df, df2[~df2["id"].isin(df["id"])]])

In [22]:
df.describe()

,score,time,id,parent,descendants,ranking,deleted,year,score_new,time_new,parent_new,descendants_new,ranking_new,deleted_new,year_new
count,26.000000,4.780000e+03,4.780000e+03,4.754000e+03,24.000000,0.0,0.0,4780.000000,24.000000,4.547000e+03,4.523000e+03,22.000000,0.0,0.0,4547.000000
mean,4.115385,1.485680e+09,1.545213e+07,1.547744e+07,5.166667,NaN,NaN,2016.602301,3.875000,1.485888e+09,1.547103e+07,5.000000,NaN,NaN,2016.608313
std,3.614714,1.237365e+08,9.532073e+06,9.527034e+06,7.069018,NaN,NaN,3.931751,3.663124,1.231144e+08,9.486637e+06,7.138094,NaN,NaN,3.910498
min,1.000000,1.175584e+09,8.462000e+03,8.407000e+03,-1.000000,NaN,NaN,2007.000000,1.000000,1.175584e+09,8.407000e+03,-1.000000,NaN,NaN,2007.000000
25%,1.250000,1.388594e+09,6.995582e+06,7.015414e+06,0.750000,NaN,NaN,2014.000000,1.000000,1.389014e+09,7.048344e+06,0.250000,NaN,NaN,2014.000000
50%,2.500000,1.504817e+09,1.519515e+07,1.524639e+07,3.000000,NaN,NaN,2017.000000,2.000000,1.504626e+09,1.520847e+07,3.000000,NaN,NaN,2017.000000
75%,6.250000,1.591880e+09,2.348823e+07,2.348918e+07,6.500000,NaN,NaN,2020.000000,4.000000,1.591172e+09,2.341565e+07,6.000000,NaN,NaN,2020.000000
max,13.000000,1.658561e+09,3.220121e+07,3.220104e+07,31.000000,NaN,NaN,2022.000000,13.000000,1.658561e+09,3.220104e+07,31.000000,NaN,NaN,2022.000000


In [23]:
rnd = df.sample(1)
print(
    f"\"{rnd['prediction'].values[0]}\" -{rnd['by'].values[0]} ({rnd['year'].values[0]})"
)

"I predict a decent chance this will be the dominant branch in under a year." -jerf (2014)


In [24]:
import json

generate = True

if generate:
    for i, d in enumerate(
        df[["id", "prediction", "by", "year"]].to_dict(orient="records")
    ):
        with open(f"../predictions/{i}", "w") as f:
            try:
                json.dump(d, f, ensure_ascii=False, indent=4)
            except UnicodeEncodeError as e:
                print(f"{i}\t{e}")


2617	'charmap' codec can't encode character '\ufff8' in position 90: character maps to <undefined>
